In [140]:
import os

In [141]:
# print the current working directory
print(os.getcwd())

s:\Shows\Shows-scripts\script_files


In [142]:
# get the list of episodes
episodes_file = os.path.join(
    os.getcwd(),
    "../shows_info_files/",
    "tmkoc episode names with numbers till 2000.txt",
)

In [157]:
# read episodes file.
# make a hashmap with type <string, int> where key is episode name and value is episode number, read each line from file
# and store it in hashmap

episodes = {}
with open(episodes_file, "r") as f:
    for line in f:
        episode_number, episode_name = line.strip().split(".", 1)
        episode_name = episode_name.strip()
        episode_name = episode_name.strip("?")
        episode_name = episode_name.replace("Taarak Mehta Ka Ooltah Chashmah - ", "")
        episodes[episode_name] = int(episode_number)

In [158]:
# print the first 100 episodes
for episode_name, episode_number in list(episodes.items())[:10]:
    print(f"{episode_number}: {episode_name}")

100: Jethalal And His Family Reach The Theatre
99: A Burglar Snatches Jethalal's Mobile
98: Jethalal Is Excited To Go For Exercise With Babita
97: Jethalal Starts Dieting
96: Babita Suggests Jethalal To Exercise
95: Winner Of Gokuldham's Cultural Dance Program
94: Special Guest In Gokuldham's Cultural Dance Program
93: Taarak Declares A Cultural Dance Program
92: Clash Of Plans
91: Dr. Hathi Gets Stuck In Auto Rickshaw


In [165]:
all_episodes_directory = [
    # "S:/tartube/Unsorted Videos",
    "S:\Shows\Taarak Mehta ka ooltah chashmah",
    "D:\Shows\Taarak Mehta Ka Ooltah Chashmah",
]


# get the list of files in the directory


all_files = []
for directory in all_episodes_directory:
    all_files.extend(os.listdir(directory))

In [166]:
episodes["Bhide And Madhavi's Dream"]

582

In [167]:
# iterate through all files in the directory, and match with the hashmap. Then rename the file with episode number, and name from hashmap
unknown_files = []
unaccounted_for_episodes = episodes.copy()
for file in all_files:
    old_file_name = file
    if file.startswith("S01E"):
        file_name_key = file.split(".", 1)[1]
        file_name_key = file_name_key.replace(".mp4", "")
        file_name_key = file_name_key.strip()
        try:
            unaccounted_for_episodes.pop(file_name_key)
        except Exception as e:
            print(
                f"Error removing {old_file_name}, {file_name_key} from unaccounted_for_episodes"
            )
            print(e)
        continue
    # format file
    file = file.strip()
    # replace name
    file = file.replace("Taarak Mehta Ka Ooltah Chashmah –", "")
    # remove extension
    file = file.replace(".mp4", "")
    file = file.strip()

    # now since we have a hash map, we can get the episode number from the hashmap
    episode_number = episodes.get(file)
    # if episode number is not found, then add the file to a list
    if episode_number is None:
        print(f"Episode number not found for {file}")
        unknown_files.append(old_file_name)
        continue

    # at this point we got a match.
    # remove the episode from the hashmap
    unaccounted_for_episodes.pop(file)

    # format the episode number to 4 digits
    episode_number = str(episode_number).zfill(4)
    # rename the file
    new_name = f"S01E{episode_number}. {file}.mp4"
    print(f"Renaming {file} to {new_name}")
    try:
        os.rename(
            os.path.join(all_episodes_directory, old_file_name),
            os.path.join(all_episodes_directory, new_name),
        )
    except Exception as e:
        print(f"Error renaming {file} to {new_name}")
        print(e)

Error removing S01E1998. Mission Saaf Safai.mp4, Mission Saaf Safai from unaccounted_for_episodes
'Mission Saaf Safai'
Error removing S01E0150. Wives Pressurize Bhide To Allow Madhavi To Join The Kitty Party.mp4, Wives Pressurize Bhide To Allow Madhavi To Join The Kitty Party from unaccounted_for_episodes
'Wives Pressurize Bhide To Allow Madhavi To Join The Kitty Party'
Error removing S01E0200. Gokuldham's  Anniversary Celebration Begins.mp4, Gokuldham's  Anniversary Celebration Begins from unaccounted_for_episodes
"Gokuldham's  Anniversary Celebration Begins"
Error removing S01E0293. Dance Till  You Drop.mp4, Dance Till  You Drop from unaccounted_for_episodes
'Dance Till  You Drop'
Error removing S01E0751. Catching The Culprit.mp4, Catching The Culprit from unaccounted_for_episodes
'Catching The Culprit'
Error removing S01E0777. Jethalal's Plan.mp4, Jethalal's Plan from unaccounted_for_episodes
"Jethalal's Plan"
Error removing S01E0876. Ladies versus Shopkeeper.mp4, Ladies versus Shop

In [168]:
print(len(unknown_files))
print(len(unaccounted_for_episodes))

0
17


In [169]:
# now for the rest of the files, we know for a fact that the names have got to do atleast something with the names in our list.
# so lets try to find a percentage match between our names and iterate through all our unaccounted for episodes

# for each episode name in unknown files, iterate through all the episode names in the hashmap and find the best match
# if the match is above a certain threshold, then rename the file

# threshold for match

threshold = 0.9
from fuzzywuzzy import fuzz

for unknown_file in unknown_files:
    max_match = 0
    best_match = None
    for episode_name in unaccounted_for_episodes.keys():
        match = fuzz.ratio(unknown_file, episode_name)
        if match > max_match:
            max_match = match
            best_match = episode_name
    if max_match > threshold:
        episode_number = unaccounted_for_episodes.pop(best_match)
        episode_number = str(episode_number).zfill(4)
        new_name = f"S01E{episode_number}. {best_match}.mp4"
        print(f"Renaming {unknown_file} to {new_name}")
        try:
            episode_directory = None
            # since we dont know the directory, lets try to rename it through all directories
            for directory in all_episodes_directory:
                try:
                    os.rename(
                        os.path.join(directory, unknown_file),
                        os.path.join(directory, new_name),
                    )
                    episode_directory = directory
                    print(f"Renamed {unknown_file} to {new_name}")
                    break
                except Exception as e:
                    print(f"Error renaming {unknown_file} to {new_name}")
                    print(e)
        except Exception as e:
            print(f"Error renaming {unknown_file} to {new_name}")
            print(e)

In [170]:
print(
    "These episodes are unaccounted for: meaning either they were not renamed somehow, or they were not downloaded"
)
for episode_name in unaccounted_for_episodes.keys():
    print(episode_name)

These episodes are unaccounted for: meaning either they were not renamed somehow, or they were not downloaded
Gokuldham's Anniversary Celebration Begins
Search For Gogi Begins
Dance Till You Drop
Bhide Takes Computer Classes From Tapu
The Disappointing Roadtrip
Daya Covers Up
Popatlal Gives A Shock
Nattu Kaka Is Back
Painting The Society
Ladies V/S Shopkeeper
The La Hula Dance
A Trip To Mahabaleshwar
The Pot Of Diamonds
A Heart-Warming Family Reunion
Where Is Bagha
Bhide And His Sakharam
Mission "Saaf Safai"


In [171]:
# rechecking the files
all_files = []
for directory in all_episodes_directory:
    all_files.extend(os.listdir(directory))

In [172]:
len(all_files)

1975

In [173]:
expected_len = 2000
if len(all_files) == expected_len:
    print(f"All files renamed successfully. Total files: {len(all_files)}")
else:
    print(
        f"Total files: {len(all_files)}. Expected {expected_len}. Please check the files. PS. make sure to check the episode numbering with the created date. Sort by episode number, if something is wrong, you will find an anomaly in created date"
    )

Total files: 1975. Expected 2000. Please check the files. PS. make sure to check the episode numbering with the created date. Sort by episode number, if something is wrong, you will find an anomaly in created date


In [174]:
# lets again find unaccounted episodes but without the hashmap ## Rigorous check
issue_episodes = {}
with open(episodes_file, "r") as f:
    for line in f:
        # find real episode name and number
        episode_number, episode_name = line.strip().split(".", 1)
        episode_name = episode_name.strip()
        episode_name = episode_name.strip("?")
        episode_name = episode_name.replace("Taarak Mehta Ka Ooltah Chashmah - ", "")

        # reconstruct the name it is supposed to have in all files
        episode_number = int(episode_number)
        episode_number = str(episode_number).zfill(4)
        episode_name = f"S01E{episode_number}. {episode_name}.mp4"

        # check if the episode is present in all files
        if episode_name not in all_files:
            print("Episode not found: ", episode_name)
            print("episode in question: ", episode_name)
            print("its number: ", episode_number)
            issue_episodes[int(episode_number)] = episode_name

Episode not found:  S01E0200. Gokuldham's Anniversary Celebration Begins.mp4
episode in question:  S01E0200. Gokuldham's Anniversary Celebration Begins.mp4
its number:  0200
Episode not found:  S01E0147. Search For Gogi Begins.mp4
episode in question:  S01E0147. Search For Gogi Begins.mp4
its number:  0147
Episode not found:  S01E0293. Dance Till You Drop.mp4
episode in question:  S01E0293. Dance Till You Drop.mp4
its number:  0293
Episode not found:  S01E0224. Daya Misses Jethalal.mp4
episode in question:  S01E0224. Daya Misses Jethalal.mp4
its number:  0224
Episode not found:  S01E0315. Bhide Takes Computer Classes From Tapu.mp4
episode in question:  S01E0315. Bhide Takes Computer Classes From Tapu.mp4
its number:  0315
Episode not found:  S01E0303. The Disappointing Roadtrip.mp4
episode in question:  S01E0303. The Disappointing Roadtrip.mp4
its number:  0303
Episode not found:  S01E0593. Daya Covers Up.mp4
episode in question:  S01E0593. Daya Covers Up.mp4
its number:  0593
Episode 

In [175]:
len(episodes)

1983

In [176]:
# so we simply have to download these episodes again.
file_with_urls = os.path.join("../shows_info_files", "tmkoc episode urls till 2000.txt")

In [177]:
urls = {}
counter = 1
# assuming each line is just the url
with open(file_with_urls, "r") as f:
    for line in f:
        urls[counter] = line
        counter += 1

In [178]:
len(urls)

2000

In [192]:
type(list(issue_episodes.keys())[0])

int

In [194]:
issue_episodes.keys()

dict_keys([200, 147, 293, 224, 315, 303, 593, 583, 519, 515, 700, 695, 748, 713, 707, 876, 847, 959, 952, 938, 902, 1064, 1028, 1011, 1497, 1592, 1687, 1682, 1752, 1812, 1998])

In [197]:
# get the links
file_name_with_desc_and_links = os.path.join(
    "../shows_info_files", "tmkoc episode with description and everything till 2k.txt"
)

episode_links = {}  # number and link, the number we can get from issue episodes
with open(file_name_with_desc_and_links, "r") as f:
    for i in range(2000):
        link = f.readline()
        thumbnail = f.readline()
        episode_number, episode_name = f.readline().strip().split(".", 1)
        duration = f.readline()
        date = f.readline()
        description = f.readline()
        if int(episode_number) in issue_episodes.keys():
            episode_links[episode_number] = link

In [198]:
episode_links

{'200': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-gokuldham-s-anniversary-celebration-begins-1000014743?watch=true\n',
 '147': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-search-for-gogi-begins-1000021338?watch=true\n',
 '293': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-dance-till-you-drop-1000014457?watch=true\n',
 '224': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-daya-misses-jethalal-1000021606?watch=true\n',
 '315': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-bhide-takes-computer-classes-from-tapu-1000014503?watch=true\n',
 '303': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-the-disappointing-roadtrip-1000014489?wa

In [202]:
for i in episode_links.values():
    print(i, end="")

https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-gokuldham-s-anniversary-celebration-begins-1000014743?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-search-for-gogi-begins-1000021338?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-dance-till-you-drop-1000014457?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-daya-misses-jethalal-1000021606?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-bhide-takes-computer-classes-from-tapu-1000014503?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-the-disappointing-roadtrip-1000014489?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-170